**D3APL: Aplicações em Ciência de Dados** <br/>
IFSP Campinas

Prof. Dr. Samuel Martins (Samuka) <br/><br/>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

# Regression Multilayer Perceptron (MLP) with Keras - V3
In this notebook, we will see how to wrap a Keras Model to be used in Sklearn environment. <br/>
We'll see:
- Fine-Tuning: `GridSearchCV`

The set up is the same from the previous version.

## 1. Set up

#### 1.1 TensorFlow + Keras

In [1]:
import tensorflow as tf
from tensorflow import keras

### 1.2 Fixing the seed for reproducibility (optional)
That's a try for reprodubility in Keras. See more on: <br/>
https://machinelearningmastery.com/reproducible-results-neural-networks-keras/ <br/>
https://www.tensorflow.org/api_docs/python/tf/random/set_seed

In [2]:
from numpy.random import seed
from tensorflow.random import set_seed

seed(42)
set_seed(42)

#### 1.3 Other imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt

#### 1.4 Loading California Housing Dataset via Scikit-learn
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html

In [4]:
X_train = np.load('./datasets/housing_train_data.npy')
y_train = np.load('./datasets/housing_train_labels.npy')
X_test = np.load('./datasets/housing_test_data.npy')
y_test = np.load('./datasets/housing_test_labels.npy')

In [5]:
print(f'X_train.shape = {X_train.shape}')
print(f'y_train.shape = {y_train.shape}\n')

print(f'X_test.shape = {X_test.shape}')
print(f'y_test.shape = {y_test.shape}\n')

X_train.shape = (16512, 8)
y_train.shape = (16512,)

X_test.shape = (4128, 8)
y_test.shape = (4128,)



#### 1.5 Normalization
For simplicity, we are going to use the `StandardScaler()` without taking into account possible problems with outliers, etc.

When normalizing the dataset before, our _cross-validation_ results will suffer a little from **data-snooping bias**.

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## 2. Wrapping Keras Model to Sklearn

### 2.1 Defining the Network Architecture
Proposed architecture for Multiclass Classification:
- Input Layer: 8 neurons
- Hidden Layer: **Number to be optimized**, ReLu
- Output Layer: 1 neuron (no activation)**

#### **Wrapping with Hyperparameters to Optimize**

##### **A function that builds a Keras Model**
Create a function that returns a **Keras model**, taking as _parameters_ things you want to verify using _cross-valdiation_ and _model selection_.

https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/

We want to _optimize_ three hyperparameters:
- The **weight initilization type** in the hidden layer;
- The **number of hidden neurons**;
- The **learning rate** of SGD.

For that, we pass three parameters to our building model function:

In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

def build_model(hidden_layers_kernel_initializer, n_hidden_neurons, learning_rate):
    model = Sequential([
        Dense(n_hidden_neurons, activation='relu', input_shape=(8,), kernel_initializer=hidden_layers_kernel_initializer),
        Dense(1)
    ])
    
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    
    return model

##### **Wrapping**
**Current alternative - SciKeras** <br/>
https://github.com/adriangb/scikeras <br/>
https://www.adriangb.com/scikeras/stable/migration.html

In [8]:
from scikeras.wrappers import KerasRegressor

# pass the function that creates a keras model, and inform the other parameters used for training, such as `epochs`, `batch_size`, etc.
model = KerasRegressor(model=build_model, hidden_layers_kernel_initializer='glorot_uniform', n_hidden_neurons=30, learning_rate=0.01, epochs=20)
model

KerasRegressor(
	model=<function build_model at 0x7f5ca78e5b80>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=20
	hidden_layers_kernel_initializer=glorot_uniform
	n_hidden_neurons=30
	learning_rate=0.01
)

### 2.2 Fine-Tuning

https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/ <br/>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Observations:
- My experiments are using a GPU drive which has limited memory. Therefore, I am using only a single job for cross-validation.
- The score functions is `neg_mean_squared_error` just to have the notion of _"the lower, the better"_. This is crucial for **fine-tunning**.

In case of GPU drivers, we can monitor its use by [_gpustat_](https://github.com/wookayin/gpustat).

On terminal, use: `gpustat -cpi`


In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'hidden_layers_kernel_initializer': ['glorot_uniform', 'he_normal'],
    'n_hidden_neurons': [10, 30],
    'learning_rate': [0.01, 0.1]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=1, scoring='neg_mean_squared_error')

grid

GridSearchCV(cv=3,
             estimator=KerasRegressor(epochs=20, hidden_layers_kernel_initializer='glorot_uniform', learning_rate=0.01, model=<function build_model at 0x7f5ca78e5b80>, n_hidden_neurons=30),
             n_jobs=1,
             param_grid={'hidden_layers_kernel_initializer': ['glorot_uniform',
                                                              'he_normal'],
                         'learning_rate': [0.01, 0.1],
                         'n_hidden_neurons': [10, 30]},
             scoring='neg_mean_squared_error')

In [10]:
grid.fit(X_train, y_train)

Epoch 1/20
344/344 [==============================] - 1s 958us/step - loss: 1.0084
Epoch 2/20
344/344 [==============================] - 0s 976us/step - loss: 0.5449
Epoch 3/20
344/344 [==============================] - 0s 956us/step - loss: 0.4989
Epoch 4/20
344/344 [==============================] - 0s 956us/step - loss: 0.4801
Epoch 5/20
344/344 [==============================] - 0s 935us/step - loss: 0.4682
Epoch 6/20
344/344 [==============================] - 0s 993us/step - loss: 0.4593
Epoch 7/20
344/344 [==============================] - 0s 954us/step - loss: 0.4526
Epoch 8/20
344/344 [==============================] - 0s 948us/step - loss: 0.4472
Epoch 9/20
344/344 [==============================] - 0s 956us/step - loss: 0.4430
Epoch 10/20
344/344 [==============================] - 0s 961us/step - loss: 0.4374
Epoch 11/20
344/344 [==============================] - 0s 980us/step - loss: 0.4357
Epoch 12/20
344/344 [==============================] - 0s 997us/step - loss: 0.4312
E

/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_regression.py", line 96, 

344/344 [==============================] - 0s 968us/step - loss: nan
Epoch 2/20
344/344 [==============================] - 0s 940us/step - loss: nan
Epoch 3/20
344/344 [==============================] - 0s 1ms/step - loss: nan
Epoch 4/20
344/344 [==============================] - 0s 970us/step - loss: nan
Epoch 5/20
344/344 [==============================] - 0s 974us/step - loss: nan
Epoch 6/20
344/344 [==============================] - 0s 950us/step - loss: nan
Epoch 7/20
344/344 [==============================] - 0s 915us/step - loss: nan
Epoch 8/20
344/344 [==============================] - 0s 917us/step - loss: nan
Epoch 9/20
344/344 [==============================] - 0s 915us/step - loss: nan
Epoch 10/20
344/344 [==============================] - 0s 979us/step - loss: nan
Epoch 11/20
344/344 [==============================] - 0s 966us/step - loss: nan
Epoch 12/20
344/344 [==============================] - 0s 922us/step - loss: nan
Epoch 13/20
344/344 [=============================

/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_regression.py", line 96, 

344/344 [==============================] - 0s 952us/step - loss: nan
Epoch 2/20
344/344 [==============================] - 0s 953us/step - loss: nan
Epoch 3/20
344/344 [==============================] - 0s 1ms/step - loss: nan
Epoch 4/20
344/344 [==============================] - 0s 903us/step - loss: nan
Epoch 5/20
344/344 [==============================] - 0s 986us/step - loss: nan
Epoch 6/20
344/344 [==============================] - 0s 1000us/step - loss: nan
Epoch 7/20
344/344 [==============================] - 0s 989us/step - loss: nan
Epoch 8/20
344/344 [==============================] - 0s 1ms/step - loss: nan
Epoch 9/20
344/344 [==============================] - 0s 997us/step - loss: nan
Epoch 10/20
344/344 [==============================] - 0s 1ms/step - loss: nan
Epoch 11/20
344/344 [==============================] - 0s 983us/step - loss: nan
Epoch 12/20
344/344 [==============================] - 0s 962us/step - loss: nan
Epoch 13/20
344/344 [==============================] 

...

/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_regression.py", line 96, 

344/344 [==============================] - 0s 930us/step - loss: nan
Epoch 2/20
344/344 [==============================] - 0s 1ms/step - loss: nan
Epoch 3/20
344/344 [==============================] - 0s 988us/step - loss: nan
Epoch 4/20
344/344 [==============================] - 0s 993us/step - loss: nan
Epoch 5/20
344/344 [==============================] - 0s 984us/step - loss: nan
Epoch 6/20
344/344 [==============================] - 0s 994us/step - loss: nan
Epoch 7/20
344/344 [==============================] - 0s 998us/step - loss: nan
Epoch 8/20
344/344 [==============================] - 0s 975us/step - loss: nan
Epoch 9/20
344/344 [==============================] - 0s 993us/step - loss: nan
Epoch 10/20
344/344 [==============================] - 0s 991us/step - loss: nan
Epoch 11/20
344/344 [==============================] - 0s 996us/step - loss: nan
Epoch 12/20
344/344 [==============================] - 0s 1ms/step - loss: nan
Epoch 13/20
344/344 [==============================]

/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/home/hisamuka/yeah/lib/python3.8/site-packages/sklearn/metrics/_regression.py", line 96, 

516/516 [==============================] - 1s 951us/step - loss: 0.7160
Epoch 2/20
516/516 [==============================] - 1s 1ms/step - loss: 0.5055
Epoch 3/20
516/516 [==============================] - 1s 1ms/step - loss: 0.6505
Epoch 4/20
516/516 [==============================] - 1s 1ms/step - loss: 0.4340
Epoch 5/20
516/516 [==============================] - 1s 1ms/step - loss: 0.4200
Epoch 6/20
516/516 [==============================] - 1s 978us/step - loss: 0.4147
Epoch 7/20
516/516 [==============================] - 1s 1ms/step - loss: 0.4029
Epoch 8/20
516/516 [==============================] - 1s 984us/step - loss: 0.3990
Epoch 9/20
516/516 [==============================] - 1s 1ms/step - loss: 0.3922
Epoch 10/20
516/516 [==============================] - 1s 1ms/step - loss: 0.3899
Epoch 11/20
516/516 [==============================] - 1s 990us/step - loss: 0.3852
Epoch 12/20
516/516 [==============================] - 1s 986us/step - loss: 0.3873
Epoch 13/20
516/516 [=====

GridSearchCV(cv=3,
             estimator=KerasRegressor(epochs=20, hidden_layers_kernel_initializer='glorot_uniform', learning_rate=0.01, model=<function build_model at 0x7f5ca78e5b80>, n_hidden_neurons=30),
             n_jobs=1,
             param_grid={'hidden_layers_kernel_initializer': ['glorot_uniform',
                                                              'he_normal'],
                         'learning_rate': [0.01, 0.1],
                         'n_hidden_neurons': [10, 30]},
             scoring='neg_mean_squared_error')

In [11]:
# best params
grid.best_params_

{'hidden_layers_kernel_initializer': 'glorot_uniform',
 'learning_rate': 0.01,
 'n_hidden_neurons': 30}

In [12]:
# get the RMSE
np.sqrt(-grid.best_score_)

0.6078680325062104

In [13]:
# get the best estimator
best_model = grid.best_estimator_
best_model

KerasRegressor(
	model=<function build_model at 0x7f5ca78e5b80>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=20
	hidden_layers_kernel_initializer=glorot_uniform
	n_hidden_neurons=30
	learning_rate=0.01
)

In [14]:
type(best_model)

scikeras.wrappers.KerasRegressor

In [15]:
# extract the true model from the wrapping
best_model = best_model.model_
best_model

In [16]:
best_model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 30)                270       
                                                                 
 dense_49 (Dense)            (None, 1)                 31        
                                                                 
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


### 2.4 Evaluating and Predicting New Samples

#### **Evaluation**
https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#evaluate

In [17]:
best_model.evaluate(X_test, y_test)  # MSE

129/129 [==============================] - 0s 780us/step - loss: 0.3784


0.37842321395874023

#### **Prediction**
https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#predict

In [18]:
y_pred = best_model.predict(X_test)
y_pred

array([[0.46552634],
       [1.5874302 ],
       [3.7443845 ],
       ...,
       [4.793019  ],
       [0.7514298 ],
       [1.678112  ]], dtype=float32)